In [2]:
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso, Ridge

import xgboost as xgb

import mlflow

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [13]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/home/ubuntu/mlops/week_2/mlruns/1', creation_time=1687511461317, experiment_id='1', last_update_time=1687511461317, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [14]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)

    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']

    df[categorical] = df[categorical].astype(str)
    return df

In [15]:
df_train = read_dataframe("../dataset/green_tripdata_2021-01.parquet")
df_val = read_dataframe("../dataset/green_tripdata_2021-02.parquet")

In [16]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dict = df_train[categorical + numerical].to_dict(orient='records')
x_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
x_val = dv.transform(val_dict)


target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [17]:
train = xgb.DMatrix(x_train, label=y_train)
valid = xgb.DMatrix(x_val, label=y_val)

In [18]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, "validation")],
            early_stopping_rounds=50
        )
        
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
    return {'loss': rmse, 'status': STATUS_OK}

In [11]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'req_alpha': hp.loguniform('req_alpha', -5, -1),
    'reg_lambda': hp.loguniform('req_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42,
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=5,
    trials=Trials()
)

[12:19:09] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[12:19:09] WARNING: ../src/learner.cc:767:           
Parameters: { "req_alpha" } are not used.

[0]	validation-rmse:17.26204                         
[1]	validation-rmse:14.29100                         
[2]	validation-rmse:12.06686                         
[3]	validation-rmse:10.43279                         
[4]	validation-rmse:9.24342                          
[5]	validation-rmse:8.39349                          
[6]	validation-rmse:7.78738                          
[7]	validation-rmse:7.36451                          
[8]	validation-rmse:7.06050                          
[9]	validation-rmse:6.84245                          
[10]	validation-rmse:6.68122                         
[11]	validation-rmse:6.57292                         
[12]	validation-rmse:6.49015                         
[13]	validation-rmse:6.42944                         
[14]	validation-rmse:6.38274